# Peer-to-Peer Autogen Assistant Agents

# Constants

In [6]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
print("Environment variables have been loaded ;-)")

deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME_MINI"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')
version = importlib.metadata.version("azure-ai-projects")
print(f"azure-ai-projects library installed version: {version}")

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Helper functions
Inspired by [Migration Guide for v0.2 to v0.4](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html)

In [7]:
def config_list_from_json(env_or_file, filter_dict):
    import json
    with open(env_or_file, 'r') as file:
        data = json.load(file)
    
    filtered_data = [
        item for item in data
        if item.get('endpoint') == filter_dict.get('endpoint') and item.get('deployment') == filter_dict.get('deployment')
    ]    
    return filtered_data


autogen_config = config_list_from_json(env_or_file, filter_dict)[0] # we take the first combination of model and endpoint

# beaware NOT to show the API KEY
print(f'AutoGen Configuration: {autogen_config["endpoint"]}, {autogen_config["deployment"]}, {autogen_config["api_version"]}, ...') 

AutoGen Configuration: https://mmoaiswc-01.openai.azure.com/, gpt-4o-2024-08-06, 2024-10-01-preview, ...


# Azure OpenAI `model client` with AutoGen

In [8]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=autogen_config["endpoint"],
    api_key=autogen_config["api_key"],
    model = autogen_config["model"],
    azure_deployment = autogen_config["deployment"],
    api_version=autogen_config["api_version"],
    seed = 41,
    temperature = 0.1,
)

# Create a copy of the dictionary  
data_to_print = model_client.dump_component().config.copy()

# Remove the 'api_key' from the copy  
del data_to_print['api_key']  
  
data_to_print

{'seed': 41,
 'temperature': 0.1,
 'model': 'gpt-4o',
 'azure_endpoint': 'https://mmoaiswc-01.openai.azure.com/',
 'azure_deployment': 'gpt-4o-2024-08-06',
 'api_version': '2024-10-01-preview'}

# [Assistant Agent definition: 0.2 vs 0.4](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html#assistant-agent)

**In `Autogen v0.2`**, we created an assistant agent as follows:
```
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='../../../config/models_list.json',
    filter_dict={ "model": {"gpt-4o-2024-05-13"} }
)

llm_config = {
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API models configurations
        "temperature": 0,  # temperature for sampling
    }

import autogen

student_agent = autogen.ConversableAgent (
    name = "Student_Agent",
    system_message = "You are a student willing to learn. You ask meaningful questions and are eager to learn more.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)
```

**In `Autogen v0.4`** it is similar, but we need to specify model_client instead of llm_config

```
autogen_config = config_list_from_json(env_or_file, filter_dict)[0]

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=autogen_config["endpoint"],
    api_key=autogen_config["api_key"],
    model = autogen_config["model"],
    azure_deployment = autogen_config["deployment"],
    api_version=autogen_config["api_version"],
    seed = 41,
    temperature = 0.1,
)

from autogen_agentchat.agents import AssistantAgent

student_agent = AssistantAgent(
    name="student_agent",
    model_client=model_client,
    system_message="""
    You are a student willing to learn. You ask meaningful and precise follow-up questions and are eager to learn more.
    When someone answers a question of yours, you always make an example to be sure you correctly understood the answer.
    Wait for your example to be answered by your conterpart.
    As soon as the answer is reasonable complete, close the conversation saying 'STUDENT IS WILLING TO TERMINATE'.
    Be concise, no more than 100 words in your replies.
    """,
)
```

### OUTPUT
```
ComponentModel(provider='autogen_ext.models.openai.OpenAIChatCompletionClient', component_type='model', version=1, component_version=1, description=None, config={'seed': 42, 'temperature': 0.1, 'model': 'gpt-4o-2024-05-13', 'api_key': '***********', 'base_url': 'https://mmoaiswc-01.openai.azure.com/'})
```

# Define two equivalent Autogen Assistant Agents

In [9]:
# AGENT 1: THE STUDENT

from autogen_agentchat.agents import AssistantAgent

student_agent = AssistantAgent(
    name="student_agent",
    model_client=model_client,
    system_message="""
    You are a student willing to learn. You ask meaningful and precise follow-up questions and are eager to learn more.
    When someone answers a question of yours, you always make an example to be sure you correctly understood the answer.
    Wait for your example to be answered by your conterpart.
    As soon as the answer is reasonable complete, close the conversation saying 'STUDENT IS WILLING TO TERMINATE'.
    Be concise, no more than 100 words in your replies.
    """,
)

In [10]:
# AGENT 2: THE STUDENT

from autogen_agentchat.agents import AssistantAgent

teacher_agent = AssistantAgent(
    name="teacher_agent",
    model_client=model_client,
    system_message="""
    You are teacher expert in may disciplines, always happy to help students or other people willing to learn.
    Your approach is to challenge a little bit any opinions of others, as Socrate was willing to do much more.
    After one challenge, wait at least one reply from your conterpart.
    When the answer is reasonable complete, wait for the counterpart reply and then close the conversation saying 'TEACHER IS WILLING TO TERMINATE'.
    Be concise, no more than 100 words in your replies.
    """,
)

# Termination Condition
It's a combination of text termination and max message termination, either of which will cause the chat to terminate.

In [11]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)

# Autogen Group Chats

## Peer-to-Peer Round Robin Group Chat

In [12]:
# The group chat will alternate between the assistant and the code executor.

from autogen_agentchat.teams import RoundRobinGroupChat
group_chat = RoundRobinGroupChat([teacher_agent, student_agent], termination_condition=termination)

stream = group_chat.run_stream(task="why does the water turns into ice under 0 celsius degrees?")

from autogen_agentchat.ui import Console
await Console(stream)

---------- user ----------
why does the water turns into ice under 0 celsius degrees?
---------- teacher_agent ----------
Water turns into ice under 0 degrees Celsius because the molecules slow down and arrange themselves into a crystalline structure. But why do you think this specific temperature is the threshold? Could there be other factors influencing this transition, such as pressure or impurities in the water?
---------- student_agent ----------
I understand that at 0 degrees Celsius, water molecules slow down enough to form a solid crystalline structure, turning into ice. For example, if I have a glass of pure water and place it in a freezer set to -5 degrees Celsius, the water should freeze into ice. Is this correct?
---------- teacher_agent ----------
Yes, that's correct. At -5 degrees Celsius, pure water will typically freeze into ice. However, consider the role of pressure and impurities. For instance, adding salt lowers the freezing point, and high pressure can also affect 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='why does the water turns into ice under 0 celsius degrees?', type='TextMessage'), TextMessage(source='teacher_agent', models_usage=RequestUsage(prompt_tokens=132, completion_tokens=53), content='Water turns into ice under 0 degrees Celsius because the molecules slow down and arrange themselves into a crystalline structure. But why do you think this specific temperature is the threshold? Could there be other factors influencing this transition, such as pressure or impurities in the water?', type='TextMessage'), TextMessage(source='student_agent', models_usage=RequestUsage(prompt_tokens=185, completion_tokens=61), content='I understand that at 0 degrees Celsius, water molecules slow down enough to form a solid crystalline structure, turning into ice. For example, if I have a glass of pure water and place it in a freezer set to -5 degrees Celsius, the water should freeze into ice. Is this correct?', type='TextMess

# HIC SUNT LEONES